In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC3
from selenium.webdriver.common.action_chains import ActionChains
from concurrent.futures import ThreadPoolExecutor
from requests.adapters import HTTPAdapter
from requests_html import HTMLSession
import time
import json
import os

op = webdriver.ChromeOptions()
# op.add_argument('--headless=new')
prefs = {
    'profile.default_content_settings.popups': 0,
    'download.default_directory': r"/home/administrator/cbs_bag_hold/data",
    'directory_upgrade': True
}
op.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(options=op)

session = HTMLSession()

def login_flo():
    driver.get("http://10.24.0.157/")
    time.sleep(5)
    username = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div/div/form/div/div[4]/input[1]")
    username.send_keys("ca.2603474")
    password = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div/div/form/div/div[4]/input[2]")
    password.send_keys("Pkkumar@1234")
    time.sleep(2)
    try:
        cross = driver.find_element(By.XPATH, "/html/body/div[4]/div/button")
        cross.click()
    except:
        print("Cross Button Failed")
    time.sleep(1)
    submit = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div/div/form/div/div[4]/div[4]/button/span")
    submit.click()
    time.sleep(3)

login_flo()
session_cookie = driver.get_cookies()

try:
    csrf_token = driver.execute_script("return document.querySelector('meta[name=csrf-token]').getAttribute('content');")
except Exception as e:
    print(f"Unable to find CSRF Token")

try:
    if csrf_token:
        session.headers.update({"csrf-token": csrf_token})
        print(csrf_token)
    else:
        print("CSRF Token not found in page")
except Exception as e:
    print(F"Unable to find csrf:")

selenium_user_agent = driver.execute_script("return navigator.userAgent;")
print(selenium_user_agent)
session.headers.update({"user-agent": selenium_user_agent})
for cookie in driver.get_cookies():
    session.cookies.set(cookie['name'], cookie['value'], domain=cookie['domain'])
  
print(session_cookie)

Cross Button Failed
Unable to find CSRF Token Message: javascript error: Cannot read properties of null (reading 'getAttribute')
  (Session info: chrome=119.0.6045.159)
Stacktrace:
#0 0x55c9db1eb5e3 <unknown>
#1 0x55c9daeae0b7 <unknown>
#2 0x55c9daeb42b5 <unknown>
#3 0x55c9daeb69b1 <unknown>
#4 0x55c9daf3f71f <unknown>
#5 0x55c9daf22ef2 <unknown>
#6 0x55c9daf3eb3d <unknown>
#7 0x55c9daf22cc3 <unknown>
#8 0x55c9daeee0e4 <unknown>
#9 0x55c9daeef0ae <unknown>
#10 0x55c9db1b1ce1 <unknown>
#11 0x55c9db1b5b7e <unknown>
#12 0x55c9db19f4b5 <unknown>
#13 0x55c9db1b67d6 <unknown>
#14 0x55c9db182dbf <unknown>
#15 0x55c9db1d9748 <unknown>
#16 0x55c9db1d9917 <unknown>
#17 0x55c9db1ea773 <unknown>
#18 0x7ff6ef9d7609 start_thread

Unable to find csrf:
Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36
[{'domain': '10.24.0.157', 'httpOnly': False, 'name': 'user', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'ca.2603474'}, {'domain': '10.2

In [6]:
def selectFacility():
    url = "http://10.24.0.157/controlledFeatures?hubId=699"
    response = session.post(url)
    print(response)


selectFacility()

<Response [405]>


In [7]:
def semiLargeTag(shipment):
    url = f"http://10.24.0.157/hub/699/shipment/{shipment}/semi-large-tag"
    data = {"is_semi_large":'true'}
    response = session.put(url , json=data)
    print(response)
    print(response.text)
    url2 = f"http://10.24.0.157/v1/hub/699/staging/suggest/{shipment}"
    data2 = {"hubId":699,"itemType":"SHIPMENT"}
    response2 = session.post(url2 , json=data2)
    print(response2)
    print(response2.text)
    response_data = json.loads(response.text)
    response_data2 = json.loads(response2.text)
    destination = response_data['data']['destination']
    shipment_id = response_data['data']['shipment_id']
    coc = response_data['data']['destination_coc']
    realGrid = response_data2['data']['stagingAreaName']

    prn = f"""CT~~CD,~CC^~CT~^XA~TA000~JSN^LT0^MNW^MTD^PON^PMN^LH0,0^JMA^PR5,5~SD30^JUS^LRN^CI27^PA0,1,1,0^XZ^XA^MMT^PW446^LL264^LS0^
    ^FO25,40^A0N,29,29^FD{shipment_id}^FS 
    ^FO25,90^A0N,50,50^FD{coc}^FS 
    ^FO25,160^A0N,29,29^FDMotherHub_YKB^FS 
    ^FO25,210^A0N,29,29^FD{destination}^FS 
    ^FO230,60^A0N,180,180^FD{realGrid}^FS^PQ1,0,1,Y^XZ
    """

    with open("a.prn" , 'w') as f:
        f.write(prn)

    os.system("lp a.prn")

            
    return print(prn)


In [8]:
if __name__ == "__main__":
    while True:
        shipment_id = input("Enter shipment ID (or type 'quit' to exit): ")
        if shipment_id.lower() == "quit":
            break
        semiLargeTag(shipment_id)

<Response [200]>
{"status":400,"message":"Unable to update shipment details","data":null}
<Response [200]>
{"status":400,"message":"null value in entry: LAYOUT_ID=null","data":null}


TypeError: 'NoneType' object is not subscriptable